# Instrumentation：基本用法


`instrumentation` 模块可用于观测和监控您的 llama-index 应用程序。它由以下核心抽象组成：

- `Event` — 表示应用程序代码执行过程中发生某个事件的时刻。
- `EventHandler` — 监听 `Event` 的发生，并在这些时刻执行代码逻辑。
- `Span` — 表示应用程序代码中特定部分的执行流程，因此包含 `Event`。
- `SpanHandler` — 负责进入、退出和丢弃（即由于错误而提前退出）`Span`。
- `Dispatcher` — 发出 `Event` 以及进入/退出/丢弃 `Span` 的信号到相应的处理程序。


在这个笔记本中，我们演示了`instrumentation`的基本使用模式：

1. 定义您自己的`EventHandler`
2. 定义您自己的`SpanHandler`，用于处理相关的`Span`类型
3. 将您的`EventHandler`和`SpanHandler`附加到所选择的调度程序（在这里，我们将其附加到根调度程序）。


In [ ]:
import nest_asyncio

nest_asyncio.apply()

### 自定义事件处理程序


In [ ]:
from llama_index.core.instrumentation.event_handlers import BaseEventHandler

定义自定义的`EventHandler`涉及子类化`BaseEventHandler`。这样做需要为抽象方法`handle()`定义逻辑。


In [ ]:
class MyEventHandler(BaseEventHandler):
    @classmethod
    def class_name(cls) -> str:
        """类名。"""
        return "MyEventHandler"

    def handle(self, event) -> None:
        """处理事件的逻辑。"""
        # 这里是你添加处理事件逻辑的地方
        print(event.dict())
        print("")
        with open("log.txt", "a") as f:
            f.write(str(event))
            f.write("\n")

### 自定义Span处理程序


`SpanHandler`也涉及子类化一个基类，在这种情况下是`BaseSpanHandler`。然而，由于`SpanHandler`需要处理一个关联的`Span`类型，如果你想处理一个新的`Span`类型，你也需要创建这个类型。


In [ ]:
from llama_index.core.instrumentation.span import BaseSpan

In [ ]:
from typing import Any, Optional
from llama_index.core.bridge.pydantic import Field
from llama_index.core.instrumentation.span.base import BaseSpan
from llama_index.core.instrumentation.span_handlers import BaseSpanHandler


class MyCustomSpan(BaseSpan):
    custom_field_1: Any = Field(...)
    custom_field_2: Any = Field(...)


class MyCustomSpanHandler(BaseSpanHandler[MyCustomSpan]):
    @classmethod
    def class_name(cls) -> str:
        """类名。"""
        return "MyCustomSpanHandler"

    def new_span(
        self, id: str, parent_span_id: Optional[str], **kwargs
    ) -> Optional[MyCustomSpan]:
        """创建一个span。"""
        # 用于创建新的MyCustomSpan的逻辑
        pass

    def prepare_to_exit_span(
        self, id: str, result: Optional[Any] = None, **kwargs
    ) -> Any:
        """准备退出span的逻辑。"""
        pass

    def prepare_to_drop_span(
        self, id: str, err: Optional[Exception], **kwargs
    ) -> Any:
        """准备放弃span的逻辑。"""
        pass

在这个笔记本中，我们将使用与`SimpleSpan`类型配合使用的`SimpleSpanHandler`。


In [ ]:
from llama_index.core.instrumentation.span_handlers import SimpleSpanHandler

### 调度器


现在我们已经有了我们的`EventHandler`和`SpanHandler`，我们可以将它们附加到一个`Dispatcher`上，该`Dispatcher`将发出`Event`和信号以开始/退出/丢弃`Span`到相应的处理程序。熟悉`logging` Python模块中的`Logger`的人可能会注意到`Dispatcher`采用了类似的接口。更重要的是，`Dispatcher`还利用了与`Logger`类似的层次结构和传播方案。具体来说，一个`dispatcher`将`Event`发送到其处理程序，并默认将这些事件传播到其父`Dispatcher`，以便其将其发送到自己的处理程序。


In [ ]:
import llama_index.core.instrumentation as instrument

dispatcher = instrument.get_dispatcher()  # 修改根调度器

In [ ]:
span_handler = SimpleSpanHandler()

dispatcher.add_event_handler(MyEventHandler())
dispatcher.add_span_handler(span_handler)

您还可以按名称获取调度程序。 仅仅是为了演示，在下面的单元格中，我们获取了在`llama_index.core`的`base.base_query_engine`子模块中定义的调度程序。


In [ ]:
qe_dispatcher = instrument.get_dispatcher(
    "llama_index.core.base.base_query_engine"
)

In [ ]:
qe_dispatcher

Dispatcher(name='llama_index.core.base.base_query_engine', event_handlers=[], span_handlers=[NullSpanHandler(open_spans={}, current_span_id=None)], parent_name='root', manager=<llama_index.core.instrumentation.dispatcher.Manager object at 0x1481ab4c0>, root_name='root', propagate=True)

In [ ]:
qe_dispatcher.parent

Dispatcher(name='root', event_handlers=[NullEventHandler(), MyEventHandler()], span_handlers=[NullSpanHandler(open_spans={}, current_span_id=None), SimpleSpanHandler(open_spans={}, current_span_id=None, completed_spans=[])], parent_name='', manager=None, root_name='root', propagate=False)

### 测试一下


In [ ]:
!mkdir -p 'data/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

data/paul_graham/paul_graham_essay.txt: No such file or directory


In [ ]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex

documents = SimpleDirectoryReader(input_dir="./data").load_data()
index = VectorStoreIndex.from_documents(documents)

In [ ]:
query_engine = index.as_query_engine()

#### 同步


In [ ]:
query_result = query_engine.query("Who is Paul?")

{'timestamp': datetime.datetime(2024, 3, 14, 15, 57, 50, 614289), 'id_': UUID('35643a53-52da-4547-b770-dba7600c9070'), 'class_name': 'QueryStartEvent'}

{'timestamp': datetime.datetime(2024, 3, 14, 15, 57, 50, 615966), 'id_': UUID('18fa9b70-6fbc-4ce7-9e45-1f292766e820'), 'class_name': 'RetrievalStartEvent'}

{'timestamp': datetime.datetime(2024, 3, 14, 15, 57, 50, 810658), 'id_': UUID('2aed6810-99a1-49ab-a6c4-5f242f1b1de3'), 'class_name': 'RetrievalEndEvent'}

{'timestamp': datetime.datetime(2024, 3, 14, 15, 57, 50, 811651), 'id_': UUID('6f8d96a2-4da0-485f-9a73-4dae2d026b48'), 'class_name': 'SynthesizeStartEvent'}

{'timestamp': datetime.datetime(2024, 3, 14, 15, 57, 50, 818960), 'id_': UUID('005e52bd-25f6-49cb-8766-5399098b7e51'), 'class_name': 'GetResponseStartEvent'}

{'timestamp': datetime.datetime(2024, 3, 14, 15, 57, 50, 823964), 'id_': UUID('52273f18-2865-42d3-a11e-acbe5bb56748'), 'class_name': 'LLMPredictStartEvent'}

{'timestamp': datetime.datetime(2024, 3, 14, 15, 57, 52, 375

#### 异步


`Dispatcher` 也适用于异步方法。


In [ ]:
query_result = await query_engine.aquery("Who is Paul?")

{'timestamp': datetime.datetime(2024, 3, 14, 15, 58, 35, 276918), 'id_': UUID('8ea98ded-91f2-45cf-b418-b92b3c7693bf'), 'class_name': 'QueryStartEvent'}

{'timestamp': datetime.datetime(2024, 3, 14, 15, 58, 35, 279006), 'id_': UUID('60ff04ce-0972-4fe1-bfaf-5ffaaea3ef4a'), 'class_name': 'RetrievalStartEvent'}

{'timestamp': datetime.datetime(2024, 3, 14, 15, 58, 35, 555879), 'id_': UUID('c29bb55e-b2e1-4637-a6cb-745a2424da90'), 'class_name': 'RetrievalEndEvent'}

{'timestamp': datetime.datetime(2024, 3, 14, 15, 58, 35, 557244), 'id_': UUID('dee202a1-f760-495e-a6f8-3644a535ff13'), 'class_name': 'SynthesizeStartEvent'}

{'timestamp': datetime.datetime(2024, 3, 14, 15, 58, 35, 564098), 'id_': UUID('c524828c-cdd7-4ddd-876a-5fda5f10143d'), 'class_name': 'GetResponseStartEvent'}

{'timestamp': datetime.datetime(2024, 3, 14, 15, 58, 35, 568930), 'id_': UUID('bd8e78ce-9a87-41a8-b009-e0694e09e0b3'), 'class_name': 'LLMPredictStartEvent'}

{'timestamp': datetime.datetime(2024, 3, 14, 15, 58, 37, 702

#### 流式处理


`Dispatcher` 也适用于支持流式传输的方法！


In [ ]:
chat_engine = index.as_chat_engine()

In [ ]:
streaming_response = chat_engine.stream_chat("Tell me a joke.")

{'timestamp': datetime.datetime(2024, 3, 14, 15, 59, 31, 345865), 'id_': UUID('1d9643ca-368b-4e08-9878-ed7682196007'), 'class_name': 'AgentChatWithStepStartEvent'}

{'timestamp': datetime.datetime(2024, 3, 14, 15, 59, 31, 346727), 'id_': UUID('c38a18e3-0c2c-43b3-a1a9-0fb2e696e627'), 'class_name': 'AgentRunStepStartEvent'}

{'timestamp': datetime.datetime(2024, 3, 14, 15, 59, 31, 348524), 'id_': UUID('9a49dd15-715c-474a-b704-b9e8df919c50'), 'class_name': 'StreamChatStartEvent'}

{'timestamp': datetime.datetime(2024, 3, 14, 15, 59, 31, 975148), 'id_': UUID('c3b5b9eb-104d-461e-a081-e65ec9dc3131'), 'class_name': 'StreamChatEndEvent'}

{'timestamp': datetime.datetime(2024, 3, 14, 15, 59, 31, 977522), 'id_': UUID('2e6c3935-8ece-49bf-aacc-de18fd79da42'), 'class_name': 'QueryStartEvent'}

{'timestamp': datetime.datetime(2024, 3, 14, 15, 59, 31, 978389), 'id_': UUID('5c43f441-d262-427f-8ef7-b3b6e6e86f44'), 'class_name': 'RetrievalStartEvent'}

{'timestamp': datetime.datetime(2024, 3, 14, 15, 59

In [ ]:
for token in streaming_response.response_gen:
    print(token, end="")

Why did the computer keep its drinks on the motherboard? Because it had too many bytes! 😄

### 使用`SimpleSpanHandler`打印基本的跟踪树



In [ ]:
span_handler.print_trace_trees()

BaseQueryEngine.query-bda10f51-e5c8-4ef8-9467-c816b0c92797 (1.762367)
└── RetrieverQueryEngine._query-35da82df-8e64-43c5-9046-11df14b3b9f7 (1.760649)
    ├── BaseRetriever.retrieve-237d1b8d-f8b1-4e0b-908c-00087f086c2c (0.19558)
    │   └── VectorIndexRetriever._retrieve-af6479b8-3210-41df-9d6f-3af31059f274 (0.194024)
    └── BaseSynthesizer.synthesize-bf923672-6e60-4015-b6fe-0d0c9d1d35e3 (1.564853)
        └── CompactAndRefine.get_response-447c173e-4016-4376-9c56-a7171ea1ddf0 (1.564162)
            └── Refine.get_response-83b1159d-d33f-401f-a76e-bc3ee5095b57 (1.557365)
                └── LLM.predict-a5ab2252-1eb1-4413-9ef0-efa14c2c3b6b (1.552019)


BaseQueryEngine.aquery-7f3daee7-540f-4189-a350-a37e7e0596d5 (1.79559)
└── RetrieverQueryEngine._aquery-cc049d88-933a-41d3-abb4-06c5bd567e45 (1.793149)
    ├── BaseRetriever.aretrieve-c1a2ae34-3916-4069-81ba-7ba9b9d7235d (0.278098)
    │   └── VectorIndexRetriever._aretrieve-ef38d533-dd12-4fa5-9879-cd885124d8aa (0.276331)
    └── BaseSynthes